In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image

In [7]:
IMAGE_PATH = '../data/images'
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 100

In [8]:
# Preprocessing function
def img_preprocessing(path):
    image = Image.open(path)
    image = image.resize((IMG_HEIGHT, IMG_WIDTH)) # Resize the image to (224, 224)
    image = image.convert('RGB') # Convert the image to RGB (in case it's grayscale or has an alpha channel)
    image = np.array(image) / 255.0
    
    return image

In [9]:
# Initialize lists to store images and labels
images = []
labels = []

# Loop through each class folder
for class_name in os.listdir(IMAGE_PATH):
    class_dir = os.path.join(IMAGE_PATH, class_name)
    if not os.path.isdir(class_dir):
        continue
    
    # Loop through each image in the class folder
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        
        # Preprocess the image
        img = img_preprocessing(image_path)
        
        # Append the image and its label to the lists
        images.append(img)
        labels.append(class_name)  # Use the folder name as the label

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [10]:
# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=42)

In [12]:
# Create an ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255,
    validation_split=0.25  # Splitting train and validation sets
)

train_generator = train_datagen.flow_from_directory(
    IMAGE_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    IMAGE_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 4050 images belonging to 90 classes.
Found 1350 images belonging to 90 classes.


In [13]:
# Load EfficientNetB0 with pre-trained weights, excluding the top layers
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

In [24]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(len(labels), activation='relu')(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

In [25]:
# Combine the base model and the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [26]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

In [28]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    callbacks=[lr_scheduler],
    epochs=EPOCHS
)

Epoch 1/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.0126 - loss: 4.5572

C:\Users\viole\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


127/127 ━━━━━━━━━━━━━━━━━━━━ 478s 4s/step - accuracy: 0.0126 - loss: 4.5571 - val_accuracy: 0.0148 - val_loss: 4.5125 - learning_rate: 1.0000e-04
Epoch 2/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 399s 3s/step - accuracy: 0.0170 - loss: 4.5160 - val_accuracy: 0.0156 - val_loss: 4.4933 - learning_rate: 1.0000e-04
Epoch 3/100
114/127 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.0142 - loss: 4.4991

KeyboardInterrupt: 

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.amp import GradScaler, autocast
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split

# Define transforms with data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(300),  # EfficientNet-B3 uses 300x300 resolution
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define transforms for validation (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize(330),  # Slightly larger than 300 for center crop
    transforms.CenterCrop(300),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
dataset = datasets.ImageFolder('../data/images', transform=train_transform)

# Split dataset into training and validation sets
train_dataset, val_dataset = train_test_split(dataset, test_size=0.3, random_state=42)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# Load pre-trained EfficientNet-B3 model
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=len(dataset.classes))

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Initialize GradScaler for mixed precision
scaler = GradScaler('cuda')

# Training loop
num_epochs = 100
best_val_loss = float('inf')
patience = 5
epochs_without_improvement = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training phase
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        # Forward pass with mixed precision
        with autocast('cuda'):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Backward pass with scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct / total

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            # Forward pass with mixed precision
            with autocast('cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = 100. * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), '../models/image_classification_model/image_classification_model.pth')
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping triggered.")
            break

    # Step the scheduler
    scheduler.step(val_loss)

print('Training complete. Best validation loss: {:.4f}'.format(best_val_loss))

Loaded pretrained weights for efficientnet-b0
Using GPU: NVIDIA GeForce RTX 3070 Laptop GPU


C:\Users\viole\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 2.8941, Train Acc: 34.02%, Val Loss: 2.2939, Val Acc: 45.00%
Epoch 2/100, Train Loss: 1.3151, Train Acc: 66.85%, Val Loss: 1.5818, Val Acc: 59.20%
Epoch 3/100, Train Loss: 0.8187, Train Acc: 78.39%, Val Loss: 1.8958, Val Acc: 52.65%
Epoch 4/100, Train Loss: 0.5523, Train Acc: 85.53%, Val Loss: 1.7386, Val Acc: 58.89%
Epoch 5/100, Train Loss: 0.3827, Train Acc: 90.26%, Val Loss: 1.9302, Val Acc: 56.67%
Epoch 6/100, Train Loss: 0.3366, Train Acc: 91.59%, Val Loss: 1.7945, Val Acc: 58.77%
Epoch 7/100, Train Loss: 0.1680, Train Acc: 95.77%, Val Loss: 1.4309, Val Acc: 66.23%
Epoch 8/100, Train Loss: 0.0909, Train Acc: 98.39%, Val Loss: 1.3849, Val Acc: 67.84%
Epoch 9/100, Train Loss: 0.0670, Train Acc: 98.92%, Val Loss: 1.3853, Val Acc: 68.02%
Epoch 10/100, Train Loss: 0.0528, Train Acc: 99.26%, Val Loss: 1.3732, Val Acc: 68.70%
Epoch 11/100, Train Loss: 0.0609, Train Acc: 99.02%, Val Loss: 1.3798, Val Acc: 69.07%
Epoch 12/100, Train Loss: 0.0468, Train Acc: 99.44%,